## Check Mediastack Dataset

#### Metadatata

In [22]:
import pandas as pd
import re

In [23]:
import sys
sys.path.append('../utils')
import functions

In [24]:
mediastack_df = pd.read_csv('../data/raw/news_sentiment_analysis.csv')

mediastack_df.head()

,Source,Author,Title,Description,URL,Published At,Sentiment,Type
0,stgnews,Bridger Palmer,Pine View High teacher wins Best in State awar...,"ST. GEORGE — Kaitlyn Larson, a first-year teac...",https://www.stgeorgeutah.com/news/archive/2024...,2024-07-12T23:45:25+00:00,positive,Business
1,Zimbabwe Mail,Staff Reporter,Businesses Face Financial Strain Amid Liquidit...,"Harare, Zimbabwe – Local businesses are grappl...",https://www.thezimbabwemail.com/business/busin...,2024-07-12T22:59:42+00:00,neutral,Business
2,4-traders,NaN,Musk donates to super pac working to elect Tru...,(marketscreener.com) Billionaire Elon Musk has...,https://www.marketscreener.com/business-leader...,2024-07-12T22:52:55+00:00,positive,Business
3,4-traders,NaN,US FTC issues warning to franchisors over unfa...,(marketscreener.com) A U.S. trade regulator on...,https://www.marketscreener.com/quote/stock/MCD...,2024-07-12T22:41:01+00:00,negative,Business
4,PLANET,NaN,Rooftop solar's dark side,4.5 million households in the U.S. have solar ...,https://www.npr.org/2024/07/12/1197961036/roof...,2024-07-12T22:28:19+00:00,positive,Business


In [25]:
functions.show_basic_info(mediastack_df)


DataFrame Shape: (3500, 8)
Number of Rows: 3500
Number of Columns: 8

Data Types of Columns:
Source          object
Author          object
Title           object
Description     object
URL             object
Published At    object
Sentiment       object
Type            object
dtype: object

Missing Values per Column:
Source            0
Author          988
Title             0
Description       0
URL               0
Published At      0
Sentiment         0
Type              0
dtype: int64

First 5 Rows of Data:
          Source          Author  \
0        stgnews  Bridger Palmer   
1  Zimbabwe Mail  Staff Reporter   
2      4-traders             NaN   
3      4-traders             NaN   
4         PLANET             NaN   

                                               Title  \
0  Pine View High teacher wins Best in State awar...   
1  Businesses Face Financial Strain Amid Liquidit...   
2  Musk donates to super pac working to elect Tru...   
3  US FTC issues warning to franchisors ove

In [26]:
functions.show_data_types(mediastack_df)

Data Types of Columns:
Source          object
Author          object
Title           object
Description     object
URL             object
Published At    object
Sentiment       object
Type            object
dtype: object


In [27]:
functions.show_missing_values(mediastack_df)


Missing Values in Columns:
Source            0
Author          988
Title             0
Description       0
URL               0
Published At      0
Sentiment         0
Type              0
dtype: int64


In [28]:
functions.check_for_duplicates(mediastack_df)


There are 737 duplicate rows in the DataFrame.


### Data Cleaning

1. Standardize column names
2. Replace empty values in the 'author' column with 'Unknown'
3. Standardize publishing date format


1. Rename columns

In [29]:
rename_dict = {
    'Source': 'source',
    'Author': 'author',
    'Title': 'title',
    'Description': 'description',
    'URL': 'url',
    'Published At': 'publish_date',
    'Sentiment': 'sentiment',
    'Type': 'content_type'
}

mediastack_df = functions.rename_columns(mediastack_df, rename_dict)

In [30]:
mediastack_df.head()

,source,author,title,description,url,publish_date,sentiment,content_type
0,stgnews,Bridger Palmer,Pine View High teacher wins Best in State awar...,"ST. GEORGE — Kaitlyn Larson, a first-year teac...",https://www.stgeorgeutah.com/news/archive/2024...,2024-07-12T23:45:25+00:00,positive,Business
1,Zimbabwe Mail,Staff Reporter,Businesses Face Financial Strain Amid Liquidit...,"Harare, Zimbabwe – Local businesses are grappl...",https://www.thezimbabwemail.com/business/busin...,2024-07-12T22:59:42+00:00,neutral,Business
2,4-traders,NaN,Musk donates to super pac working to elect Tru...,(marketscreener.com) Billionaire Elon Musk has...,https://www.marketscreener.com/business-leader...,2024-07-12T22:52:55+00:00,positive,Business
3,4-traders,NaN,US FTC issues warning to franchisors over unfa...,(marketscreener.com) A U.S. trade regulator on...,https://www.marketscreener.com/quote/stock/MCD...,2024-07-12T22:41:01+00:00,negative,Business
4,PLANET,NaN,Rooftop solar's dark side,4.5 million households in the U.S. have solar ...,https://www.npr.org/2024/07/12/1197961036/roof...,2024-07-12T22:28:19+00:00,positive,Business
